In [73]:
import requests
# import schedule
import time
import asyncio
from datetime import datetime
from datetime import timedelta


dry_hours = 0
API_KEY = 'f18a4a397e68da9186dfd4d31a37b44e'
ZIP = 13820

city_name = 'Oneonta'
state_code = 'NY'
country_code = 'USA'




In [ ]:
def get_lat_lon(city_name, state_code, country_code='USA'):
    zip_encode_url = f'http://api.openweathermap.org/geo/1.0/direct?q={city_name},{state_code},{country_code}&limit={3}&appid={API_KEY}'
    zip_resp = requests.get(zip_encode_url)
    zip_data = zip_resp.json() 
    
    zip_lat = zip_data[0]['lat']
    zip_lon = zip_data[0]['lon']

    return zip_lat, zip_lon 

def get_rain_status():
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?zip={ZIP},us&appid={API_KEY}&units=imperial"
        response = requests.get(url)
        data = response.json()
        rain = data.get("rain", {})
        if "1h" in rain and rain["1h"] > 0:
            print(f"{datetime.now()}: Rain detected ({rain['1h']} mm).")
            return True
        print(f"{datetime.now()}: No rain detected.")
        return False
    except Exception as e:
        print(f"Error checking weather: {e}")
        return False
    
def get_watering_score(zip_lat, zip_lon):
    try: 
        future_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={zip_lat}&lon={zip_lon}&appid={API_KEY}'
        future_resp = requests.get(future_url) 
        future_data  = future_resp.json()
        future_resp 

        codes_thunderstorm = [i for i in range(200,300)]
        codes_drizzle = [i for i in range(300,400)]
        codes_rain = [i for i in range(500,600)]

        thunderstorm_recs = 0
        drizzle_recs = 0
        rain_recs = 0 

        watering_score = 0

        forecast_focus  = [datum for datum in  future_data['list']  \
                        if int( (datetime.fromtimestamp(datum['dt']) -datetime.now() ).total_seconds()/3600 ) <= 12 \
                        ]
        forecast_focus

        for item in forecast_focus:
            # check_time = datetime.fromtimestamp(datum['dt'])-timedelta(hours=9)   #wi
            delta =  datetime.fromtimestamp(item['dt']) -datetime.now()
            minutes_until_forecast = int(delta.total_seconds() )/60
            hours_until_forecast = minutes_until_forecast / 60
            
            rain_events = []
            
            print(f'future - {minutes_until_forecast} minutes aka {hours_until_forecast} hours') 
            print(f"{datetime.fromtimestamp(item['dt'] )} -- {item['weather'][0]['id']} ")
            print(item['weather'])
            print(item['weather'][0]['main'])
            print('------------------------------') 
            # if datum['weather'][0]['main'] == 'Rain':
                # rain_recs += 1 
            
            if item['weather'][0]['id'] in codes_thunderstorm:
                thunderstorm_recs += 1
                rain_events.append(hours_until_forecast)
            elif item['weather'][0]['id'] in codes_drizzle:
                drizzle_recs +=1
                rain_events.append(hours_until_forecast)
            elif item['weather'][0]['id'] in codes_rain:
                rain_recs +=1
                rain_events.append(hours_until_forecast)


            print(rain_events)
            
        if len(rain_events) == 0: 
            next_rain = 12
        else: 
            next_rain = min(rain_events) 

        print(next_rain)


        # if rain_recs+thunderstorm_recs+drizzle_recs <=1 and next_rain <=3: 
        if next_rain >9:
            watering_score += 3
        if next_rain >6:
            watering_score +=2
        if next_rain >3:
            watering_score +=1
        if thunderstorm_recs >=1 and next_rain <=3:
            watering_score -=2
        if drizzle_recs >2 and next_rain<=3:
            watering_score -=1
        if rain_recs >1 and next_rain <=3:
            watering_score -=1 
          
        if watering_score >1:
            print("Fire the sprinklers")
        else: 
            print("Rain is a comin' ")
        
        return watering_score
        
    except Exception as e:
        print(f"Error getting watering score: {e}")
        return False 

# async def activate_plug():
#     try:
#         client = TapoApiClient(config.TAPO_IP, config.TAPO_USERNAME, config.TAPO_PASSWORD)
#         await client.login()
#         print(f"{datetime.now()}: Turning plug ON")
#         await client.set_device_on()
#         await asyncio.sleep(300)  # 5 minutes
#         print(f"{datetime.now()}: Turning plug OFF")
#         await client.set_device_off()
#     except Exception as e:
#         print(f"Error controlling plug: {e}")

# def check_weather_and_trigger():
#     global dry_hours
#     rained = get_rain_status()
#     if rained:
#         dry_hours = 0
#     else:
#         dry_hours += 1
#         print(f"{datetime.now()}: Dry for {dry_hours} hour(s)")
#         if dry_hours >= config.DRY_HOURS_THRESHOLD:
#             dry_hours = 0
#             asyncio.run(activate_plug())

def main():
    # schedule.every(60).minutes.do(check_weather_and_trigger)
    print("Starting garden irrigation watcher...")
    # check_weather_and_trigger()  # initial check
    lat, lon = get_lat_lon('Oneonta','NY') 
    
    while True:    
        if get_rain_status():
            print('rain reported') 
        
        else: 
            if get_watering_score(lat, lon)>=1:
                print("FIre the sprinklers") 
                print("Plug ON")
                print("Wait") 
                print("Plug Off:")
   
        time.sleep(60)

if __name__ == "__main__":
    main()

Starting garden irrigation watcher...
2025-06-27 12:40:42.600194: No rain detected.
future - 79.28333333333333 minutes aka 1.321388888888889 hours
2025-06-27 14:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 259.28333333333336 minutes aka 4.321388888888889 hours
2025-06-27 17:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 439.28333333333336 minutes aka 7.321388888888889 hours
2025-06-27 20:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 619.2833333333333 minutes aka 10.321388888888889 hours
2025-06-27 23:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]
Clouds
------------------------------
[]
12
Fire the sprinklers
FIre the sprinklers
Plug ON
Wait
Plug Off:

In [68]:
get_rain_status()

2025-06-27 12:14:25.503546: No rain detected.


False

In [71]:
lat, lon= get_lat_lon('Oneonta','NY')
print(lat, lon)
get_watering_score(lat, lon)

42.453492 -75.0629531
future - 104.48333333333333 minutes aka 1.7413888888888889 hours
2025-06-27 14:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 284.48333333333335 minutes aka 4.741388888888889 hours
2025-06-27 17:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 464.48333333333335 minutes aka 7.741388888888889 hours
2025-06-27 20:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 644.4833333333333 minutes aka 10.741388888888888 hours
2025-06-27 23:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]
Clouds
------------------------------
[]
12
Fire the sprinklers


In [33]:
# ZIP = '56345'
try:
        url = f"https://api.openweathermap.org/data/2.5/weather?zip={ZIP},us&appid={API_KEY}&units=imperial"
        response = requests.get(url)
        data = response.json()
        rain = data.get("rain", {})
        if "1h" in rain and rain["1h"] > 0:
            print(f"{datetime.now()}: Rain detected ({rain['1h']} mm).")
            
        print(f"{datetime.now()}: No rain detected.")
        
except Exception as e:
    print(f"Error checking weather: {e}")
    

2025-06-27 07:55:03.508460: No rain detected.


In [34]:
data 

{'coord': {'lon': -75.0491, 'lat': 42.4625},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 58.71,
  'feels_like': 58.75,
  'temp_min': 57.65,
  'temp_max': 60.28,
  'pressure': 1023,
  'humidity': 95,
  'sea_level': 1023,
  'grnd_level': 967},
 'visibility': 10000,
 'wind': {'speed': 2.75, 'deg': 122, 'gust': 7.76},
 'clouds': {'all': 100},
 'dt': 1751025192,
 'sys': {'type': 2,
  'id': 2001760,
  'country': 'US',
  'sunrise': 1751016298,
  'sunset': 1751071286},
 'timezone': -14400,
 'id': 0,
 'name': 'Oneonta',
 'cod': 200}

In [35]:
wind = data.get('wind')
wind 

{'speed': 2.75, 'deg': 122, 'gust': 7.76}

In [36]:
city_name = 'Oneonta'
state_code = 'NY'
country_code = 'USA'
zip_encode_url = f'http://api.openweathermap.org/geo/1.0/direct?q={city_name},{state_code},{country_code}&limit={3}&appid={API_KEY}'

zip_resp = requests.get(zip_encode_url)
zip_data = zip_resp.json() 

zip_data
zip_lat = zip_data[0]['lat']
zip_lon = zip_data[0]['lon']



In [49]:
future_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={zip_lat}&lon={zip_lon}&appid={API_KEY}'
future_resp = requests.get(future_url) 
future_data  = future_resp.json()
future_resp 

<Response [200]>

[{'dt': 1751047200,
  'main': {'temp': 291.47,
   'feels_like': 291.39,
   'temp_min': 291.47,
   'temp_max': 293.31,
   'pressure': 1022,
   'sea_level': 1022,
   'grnd_level': 965,
   'humidity': 78,
   'temp_kf': -1.84},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': {'all': 98},
  'wind': {'speed': 5.74, 'deg': 173, 'gust': 7.48},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2025-06-27 18:00:00'},
 {'dt': 1751058000,
  'main': {'temp': 294.41,
   'feels_like': 294.47,
   'temp_min': 294.41,
   'temp_max': 296.34,
   'pressure': 1021,
   'sea_level': 1021,
   'grnd_level': 964,
   'humidity': 72,
   'temp_kf': -1.93},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'clouds': {'all': 72},
  'wind': {'speed': 5.54, 'deg': 175, 'gust': 7.84},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2025-06-27 21:00:00

In [ ]:

codes_thunderstorm = [i for i in range(200,300)]
codes_drizzle = [i for i in range(300,400)]
codes_rain = [i for i in range(500,600)]

thunderstorm_recs = 0
drizzle_recs = 0
rain_recs = 0 

watering_score = 0

forecast_hours = 12

forecast_focus  = [datum for datum in  future_data['list']  \
                   if int( (datetime.fromtimestamp(datum['dt']) -datetime.now() ).total_seconds()/3600 ) <= 12 \
                   ]
forecast_focus

for item in forecast_focus:
    # check_time = datetime.fromtimestamp(datum['dt'])-timedelta(hours=9)   #wi
    delta =  datetime.fromtimestamp(item['dt']) -datetime.now()
    minutes_until_forecast = int(delta.total_seconds() )/60
    hours_until_forecast = minutes_until_forecast / 60
    
    rain_events = []
    
    print(f'future - {minutes_until_forecast} minutes aka {hours_until_forecast} hours') 
    print(f"{datetime.fromtimestamp(item['dt'] )} -- {item['weather'][0]['id']} ")
    print(item['weather'])
    print(item['weather'][0]['main'])
    print('------------------------------') 
    # if datum['weather'][0]['main'] == 'Rain':
        # rain_recs += 1 
    
    if item['weather'][0]['id'] in codes_thunderstorm:
        thunderstorm_recs += 1
        rain_events.append(hours_until_forecast)
    elif item['weather'][0]['id'] in codes_drizzle:
        drizzle_recs +=1
        rain_events.append(hours_until_forecast)
    elif item['weather'][0]['id'] in codes_rain:
        rain_recs +=1
        rain_events.append(hours_until_forecast)


    print(rain_events)
    
if len(rain_events) == 0: 
    next_rain = 12
else: 
    next_rain = min(rain_events) 

print(next_rain)

# if rain_recs+thunderstorm_recs+drizzle_recs <=1 and next_rain <=3: 
if next_rain >3:
    print("Fire the Sprinklers!")
else: 
    print("Rain is a comin' ")

future - 116.9 minutes aka 1.9483333333333335 hours
2025-06-27 14:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 296.9 minutes aka 4.948333333333333 hours
2025-06-27 17:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 476.9 minutes aka 7.948333333333333 hours
2025-06-27 20:00:00 -- 803 
[{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
Clouds
------------------------------
[]
future - 656.9 minutes aka 10.948333333333332 hours
2025-06-27 23:00:00 -- 804 
[{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]
Clouds
------------------------------
[]
0
Fire the Sprinklers!


In [25]:
len(future_data['list'])

40

In [22]:
list = future_data.get('list')
list


[{'dt': 1750950000,
  'main': {'temp': 294.21,
   'feels_like': 294.59,
   'temp_min': 294.04,
   'temp_max': 294.21,
   'pressure': 1019,
   'sea_level': 1019,
   'grnd_level': 963,
   'humidity': 85,
   'temp_kf': 0.17},
  'weather': [{'id': 500,
    'main': 'Rain',
    'description': 'light rain',
    'icon': '10d'}],
  'clouds': {'all': 88},
  'wind': {'speed': 1.18, 'deg': 101, 'gust': 1.87},
  'visibility': 10000,
  'pop': 0.39,
  'rain': {'3h': 0.19},
  'sys': {'pod': 'd'},
  'dt_txt': '2025-06-26 15:00:00'},
 {'dt': 1750960800,
  'main': {'temp': 296.42,
   'feels_like': 296.84,
   'temp_min': 296.42,
   'temp_max': 297.48,
   'pressure': 1018,
   'sea_level': 1018,
   'grnd_level': 961,
   'humidity': 78,
   'temp_kf': -1.06},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': {'all': 94},
  'wind': {'speed': 0.82, 'deg': 86, 'gust': 3.63},
  'visibility': 10000,
  'pop': 0.22,
  'sys': {'pod': 'd'},
  'dt_txt

In [8]:
import smtplib

from email.message import EmailMessage
from datetime import datetime

'klen claj wsgi qenc'
time = datetime.now()

message_to_send = f"""Time: {time} 
Status set to :Test
Custom Message: 

"""

msg = EmailMessage()
msg["Subject"] = f"IrrigationUpdate" 
msg.set_content(message_to_send)
msg["From"] = "CustomAlerts123321@gmail.com"
msg["To"] = "tiedieguy2k7@gmail.com"

with smtplib.SMTP("smtp.gmail.com", 587) as smtp:
    # smtp.set_debuglevel(1)
    smtp.ehlo()
    smtp.starttls()
    smtp.login("customalerts123321", "klenclajwsgiqenc")
    smtp.send_message(msg) 

In [ ]:
from datetime import datetime
from time import sleep
t1 = datetime.now()

sleep(60) 

t2 = datetime.now() 

test = t2-t1

print(test) 

-1 day, 23:58:59.998263


In [20]:
test = t2-t1

print(test) 
if test.seconds/60 <= 2:
    print(1)
else:
    print(2) 

0:01:00.001737
1


In [ ]:

try: 
    with open("last_irrigation.txt", "w") as li:
        text =li.read()
        if len(li.read) >0:
            print(text)
        else:
            print('No Text')
except:
    print('no file')
    with open('LastIrrigation.txt','a') as f:
        f.write(f"{datetime.now()}")

    

no file


TypeError: write() argument must be str, not datetime.datetime